<a href="https://colab.research.google.com/github/djtroyal/image2stl/blob/main/image2stl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install Pillow
!pip install numpy-stl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import numpy as np
from PIL import Image
from stl import mesh
import requests
import os

# Ask for user input
img_source = input("Enter image URL or file path: ")

# Load image and convert to grayscale
if img_source.startswith("http"):
    img = Image.open(requests.get(img_source, stream=True).raw).convert('L')
else:
    img = Image.open(img_source).convert('L')

# Resize image while maintaining aspect ratio
max_size = 500
width, height = img.size
if width > height:
    new_width = max_size
    new_height = int(max_size * height / width)
else:
    new_height = max_size
    new_width = int(max_size * width / height)
img = img.resize((new_width, new_height))

# Flip image along vertical axis
img = img.transpose(method=Image.FLIP_LEFT_RIGHT)

# Rotate image 180 degrees along z-axis
img = img.rotate(180)

# Convert image to numpy array
img_array = np.array(img)

# Create mesh
z_scale = 50 #Update this value to make the z-axis more or less pronounced
x_scale = 1
y_scale = 1
z_min = np.min(img_array)
z_max = np.max(img_array)
z_span = z_max - z_min
vertices = np.zeros((new_height, new_width, 3))
for i in range(new_height):
    for j in range(new_width):
        vertices[i][j][0] = j * x_scale
        vertices[i][j][1] = i * y_scale
        vertices[i][j][2] = (img_array[i][j] - z_min) * z_scale / z_span
faces = []
for i in range(new_height - 1):
    for j in range(new_width - 1):
        vertex1 = vertices[i][j]
        vertex2 = vertices[i+1][j]
        vertex3 = vertices[i+1][j+1]
        vertex4 = vertices[i][j+1]
        face1 = [vertex1, vertex2, vertex3]
        face2 = [vertex1, vertex3, vertex4]
        faces.append(face1)
        faces.append(face2)
mesh_data = mesh.Mesh(np.zeros(len(faces), dtype=mesh.Mesh.dtype))
for i, f in enumerate(faces):
    for j in range(3):
        mesh_data.vectors[i][j] = f[j]
        
# Export mesh to STL file
source_name = os.path.splitext(os.path.basename(img_source))[0]
stl_file_name = source_name + '.stl'
mesh_data.save(stl_file_name)
print(f"STL file saved as {stl_file_name}")


Enter image URL or file path: https://external-preview.redd.it/kIOROsGqy5KqIfpf8w7_FhkJvuGbMvqS0sPig_d_wco.jpg?auto=webp&v=enabled&s=12fa0577c8fc9d68d246d91d96900eba3b967f59
STL file saved as kIOROsGqy5KqIfpf8w7_FhkJvuGbMvqS0sPig_d_wco.stl
